In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import glob
import os
import numpy as np
from PIL import Image
# from google.colab import drive
import torchvision.models as models
from tqdm import tqdm

In [2]:
# upload images from googledrive
# drive.mount('/content/gdrive')
# %cd ./gdrive/MyDrive/2021-DLCV-HW/HW1/
# !unzip hw1_data.zip

In [3]:
class HWIMGS(Dataset):
  def __init__(self, root, transform=None):
    " initial the dataset "
    self.image = None
    self.label = None
    self.filenames = []
    self.root = root;
    self.transform = transform

    #read filenames
    filenames = glob.glob(root+'/*.png')
    for fn in filenames:
        #store data, label
        reg = fn
        
#         colab:/*.png
#         jupyter:\*.png
        reg = fn.split('\\')[-1].split('_')[0]
#         print(reg)
        reg = int(reg)
        self.filenames.append((fn, reg))

    self.len = len(filenames)

  def __getitem__(self, index):
    " get a sample from the dataset "
    img_fn, label = self.filenames[index]
    # if torch.cuda.is_available():
    #   img_fn, label = img_fn.cuda(), label.cuda()
    image = Image.open(img_fn)
    if self.transform is not None:
      image = self.transform(image)
    
    return image, label
  
  def __len__(self):
    " Total number of sampler in the dataset "
    return self.len

In [4]:
from torchvision import transforms
trainset = HWIMGS(root='p1_data/train_50', transform=transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
]))

valset = HWIMGS(root='p1_data/val_50', transform=transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
]))
print(len(trainset))
print(len(valset))

22500
2500


In [5]:
trainset_loader = DataLoader(trainset, batch_size=4, shuffle=True)
valset_loader = DataLoader(valset, batch_size=4, shuffle=False)

In [6]:
# dataiter = iter(trainset_loader)
# images, labels = dataiter.next()

In [7]:
# print('Image tensor in each batch:', images.shape, images.dtype)
# print('Image tensor in each batch:', labels.shape, labels.dtype)

NameError: name 'images' is not defined

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [9]:
vgg16 = models.vgg16(pretrained=True)
vgg16.classifier[6]= nn.Sequential(
    nn.Dropout(0.2),
    nn.Linear(4096, 50)
)
# vgg16.classifier[6]=nn.Linear(4096,50)
vgg16 = vgg16.to(device)

In [10]:
# from torchsummary import summary
# resnet18 = models.resnet18(pretrained=True)
# # print(resnet18)
# # summary(resnet18, (3, 224, 224))
# resnet18.fc= nn.Sequential(
    
#     nn.Linear(512, 256),
#     nn.ReLU(),
#     nn.BatchNorm1d(256),
#     nn.Dropout(0.3),
    
# #     nn.Linear(256, 100),
# #     nn.ReLU(),
# #     nn.BatchNorm1d(100),
# #     nn.Dropout(0.3),
    
#     nn.Linear(256, 50)
# )
# resnet18 = resnet18.to(device)
# summary(resnet18, (3, 224, 224))

In [11]:
# from torchsummary import summary
# resnet101 = models.resnet101(pretrained=True)

# resnet101.fc = nn.Sequential(
    
# #     nn.Linear(2048, 256),
# #     nn.ReLU(),
# #     nn.BatchNorm1d(256),
# #     nn.Dropout(0.3),
    
# #     nn.Linear(256, 100),
# #     nn.ReLU(),
# #     nn.BatchNorm1d(100),
# #     nn.Dropout(0.3),
    
#     nn.Linear(2048, 50)
# )
# resnet101 = resnet101.to(device)
# summary(resnet101, (3, 224, 224))
# print(resnet101)

In [12]:
def train(model, epoch, log_interval=100):
    optimizer = optim.Adam(model.parameters(), lr=3e-4)
    scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
    criterion = nn.CrossEntropyLoss()
    model.train()  # Important: set training mode
    
    iteration = 0
    for ep in range(epoch):
        for batch_idx, (data, target) in enumerate(tqdm(trainset_loader), 1):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            
            if iteration % log_interval == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    ep+1, batch_idx * len(data), len(trainset_loader.dataset),
                    100. * batch_idx / len(trainset_loader), loss.item()))
            iteration += 1
        val(model, scheduler) # Evaluate at the end of each epoch

In [13]:
def val(model, scheduler):
    criterion = nn.CrossEntropyLoss()
    model.eval()  # Important: set evaluation mode
    val_loss = 0
    correct = 0
    with torch.no_grad(): # This will free the GPU memory used for back-prop
        for data, target in valset_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            val_loss += criterion(output, target).item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    val_loss /= len(valset_loader.dataset)
    print('\nVal set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        val_loss, correct, len(valset_loader.dataset),
        100. * correct / len(valset_loader.dataset)))
    scheduler.step()

In [ ]:
train(vgg16, epoch=5)

  0%|                                                                               | 1/5625 [00:01<2:06:13,  1.35s/it]

Train Epoch: 1 [4/22500 (0%)]	Loss: 4.193804


  2%|█▍                                                                           | 101/5625 [01:05<1:16:10,  1.21it/s]

Train Epoch: 1 [404/22500 (2%)]	Loss: 3.864074


  4%|██▊                                                                          | 201/5625 [02:09<1:14:39,  1.21it/s]

...

 43%|█████████████████████████████████▎                                            | 2401/5625 [25:43<44:38,  1.20it/s]

Train Epoch: 1 [9604/22500 (43%)]	Loss: 3.820957


 44%|██████████████████████████████████▋                                           | 2501/5625 [26:47<43:27,  1.20it/s]

Train Epoch: 1 [10004/22500 (44%)]	Loss: 3.837223


 45%|██████████████████████████████████▊                                           | 2510/5625 [26:52<33:08,  1.57it/s]

In [ ]:
torch.save(model.resnet50(), './resnet18_1.pth')